# Ch 5. 텐서보드와 모델 재사용

목표 : <br>
오늘의 공부 목표는 Ch 5의 텐서보드 사용 및 모델 재사용 코드를 숙달하는 것입니다. 숙달의 의미는 안 보고도 짤 수 있는 걸 의미합니다. 우선 P.74부터 존재하는 코드를 작성한 후 스스로 주석을 다는 작업을 하겠습니다.(a.m. 9:27 시작)
<br>
참조 : [골빈 해커의 3분 딥러닝](https://github.com/golbin/TensorFlow-Tutorials) 소스 코드를 학습용으로 사용. 

In [1]:
# 1. Import modules
# 신경망 구축 및 텐서보드 사용+csv파일 로드 및 전치 사용을 위해
# Tensorflow와 Numpy를 import함.
import tensorflow as tf
import numpy as np

# 2. Load .csv data
# './filename.format'에서 ./는 현재 폴더 혹은 디렉토리를 가리킨다.
# delimiter는 기본값은 공백(white space)이고, string을 개별 값들로 분리하는 기준.
# unpack은 반환할 array를 transpose할지 말지 정하는 것이다. True일 경우 transpose되어 반환된다.
data = np.loadtxt('./data.csv', delimiter=',',
                 unpack=True, dtype='float32')

# unpack에 의해 transpose되었으므로 데이터를 x와 y로 분류할 때 한 번 더
# transpose를 취해준다.
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

# 3. 모델을 저장할 때 쓸 변수 선언
# 학습 횟수를 카운트하는 변수임
# trainable은 이 변수가 학습에 직접 사용될 것인지 아닌지를 정해 줌. Default는 True.
# 학습에 직접 사용될 경우 해당 변수는 Optimizer class가 사용함.
global_step = tf.Variable(0, trainable=False, name='global_step')

# 4. Model
# 입력 레이어
# x_data와 y_data를 담을 그릇
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 첫 번재 레이어
# tf.random uniform(shape, minval,maxval,dtype,seed,name)
# -1., 1.일 경우 최소 -1에서 최대 1 사이의 수가 생성된다.
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X,W1))

# 두 번째 레이어
W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

# 출력 레이어
W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

# Cost for training
# tf.nn.softmax_cross_entropy_with_logits : logits와 labels 사이의 소프트맥스 크로스 엔트로피를 계산해준다.(소프트맥스 크로스 엔트로피 공부할 것 [  ])
# 보통 labels는 원핫인코딩과 같은 형태를 입력하며, 스칼라값 혹은 1 class값만을 넣을 때는 sparse_softmax_cross_entropy_with_logits를 사용하라.
# labels와 logits는 shape과 datatype이 같아야 한다. 예를 들어 [batch_size, num_classes] 같이.
# tf.reduce_mean(tensor, axis)은 원소들의 평균을 구해 반환함으로써 Tensor의 차원을 축소해준다.
# axis = 0일 때, 그냥 모든 원소의 평균을 구해서 반환한다.
# axis = 1일 때, column끼리 평균을 내서 반환한다(2차 행렬에서의 그 column).
# axis = 2일 때, row끼리 평균을 내서 반환한다(2차 행렬에서의 그 row).
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate =0.01) # Question : learning rate를 decay하려면 어떻게 해야 할까? [  ]
train_op = optimizer.minimize(cost, global_step=global_step)

(a.m. 10:31) 일단 모델 코드까지만 주석을 달아봤다.
현재 더 찾아보고 싶은 부분은<br>
1. 소프트맥스 크로스 엔트로피
2. tf.train 클래스
3. tf.train Optimizer에서 learning rate를 decay하는 방법

2018년 1월 21일 일요일

In [2]:
#########
# 신경망 모델 학습
######
sess = tf.Session()
# 모델을 저장하고 불러오는 API인 tf.train.Saver를 초기화합니다.
# global_variables 함수를 통해 앞서 정의하였던 변수들을 저장하거나 불러올 변수들로 설정합니다.
# tf.Variable이나 tf.get_variable을 쓰면 생성되는 변수들은 자동적으로 graphkeys에 저장이 되는데,
# tf.global_variables는 그렇게 저장된 변수들의 리스트를 반환한다.
# 참고로 global variable이란 분산 환경에서 모든 machine에 사용할 수 있는 변수를 지칭한다.
saver = tf.train.Saver(tf.global_variables())

# get_checkpoint_state는 디렉토리에 있는 모델 파일의 CheckPointState의 proto를 반환한다. 없으면 아무것도 반환하지 않는다.
ckpt = tf.train.get_checkpoint_state('./model')
# ckpt의 모델 폴더에 체크포인트가 존재하면
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    # 세션에 체크포인트 상태를 복원한다.
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    # 존재하지 않으면 전역 변수를 초기화한다.
    sess.run(tf.global_variables_initializer())

2018년 1월 22일 월요일

In [6]:
# 최적화 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    # 적절한 시점에 저장할 값들을 수집하고 저장합니다.
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)


('Step: 3, ', 'Cost: 1.168')


TypeError: Fetch argument None has invalid type <type 'NoneType'>